In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
import math
import string
import matplotlib.pyplot as plt

#Loading the dataset

Mount Drive

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


Load the dataset from the csv file, the bad lines are skipped and the null values are removed.

In [ ]:
#Load the dataset as csv, skipping the bad lines
csv = pd.read_csv("drive/Shareddrives/BPM PROJECT/Dataset/amazon_reviews_reduced.csv", on_bad_lines="skip")

#Drop the null values
csv.dropna(inplace=True)

#Check the number of the reviews for each product in the reduced dataset
csv_grouped = csv.groupby(by="product_id").count().sort_values(by="review_body")
csv_grouped

,Unnamed: 0,marketplace,customer_id,review_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
product_id,,,,,,,,,,,,,,,
B0001FTVEK,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792,8792
B0012S4APK,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358,9358
B003EM8008,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766,9766
B0002L5R78,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166,11166
B003L1ZYYM,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333,15333


In [ ]:
csv.head(2)

,Unnamed: 0,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,59,US,7260378,R9XIRRPMX4Q64,B003L1ZYYM,617978254,AmazonBasics High-Speed HDMI Cable - 6.5 Feet ...,Electronics,5,0,0,N,Y,Five Stars,Does the job.,2015-08-31
1,141,US,24439808,R17U6AU06HR16Q,B0001FTVEK,796776495,Sennheiser On-Ear 926MHz Wireless RF Headphone...,Electronics,5,0,0,N,Y,Five Stars,Great product.,2015-08-31


# Dataset creation

For the test of the models we'll use just the reviews of the product with product_id **B003L1ZYYM**, the features that we use are the **review_body**, **review_headline** used as label during the fine tuning process, **star_rating**, **helpful_votes** and **review_date**

In [ ]:
csv_r = csv[csv["product_id"] == "B003L1ZYYM"]
csv_r = csv_r[["review_body", "review_headline", "star_rating","helpful_votes","review_date"]]
csv_r.reset_index(inplace=True, drop=True)
csv_r

,review_body,review_headline,star_rating,helpful_votes,review_date
0,Does the job.,Five Stars,5,0,2015-08-31
1,Cool,Five Stars,5,0,2015-08-31
2,They work fine.,Five Stars,5,0,2015-08-31
3,I love it,Five Stars,5,0,2015-08-31
4,"Buenas, excelente cable.",Five Stars,5,0,2015-08-31
...,...,...,...,...,...
15328,This is the HDMI cable to get right now: an HD...,A Basic Cable at a Good Price,5,297,2010-11-01
15329,I connected my Blu Ray disc player to my HDTV ...,Easy and lightweight HDMI cable does it all.,5,5,2010-11-01
15330,"Just unpacked my cable,it is really very high ...",High quality,5,1,2010-11-01
15331,I have purchased quite a few AmazonBasics item...,It works.,5,12,2010-11-01


In [ ]:
# csv_r["review_weight"] = math.log(csv_r["helpful_votes"]+1)*csv_r["star_rating"]

csv_r["review_weight"] = csv_r.apply(lambda x: math.log(x["helpful_votes"]+1)*x["star_rating"], axis=1)

In [ ]:
csv_r[csv_r["review_weight"] != 0]

,review_body,review_headline,star_rating,helpful_votes,review_date,review_weight
24,This cord works very well. A lot of other chea...,5 Stars,5,1,2015-08-30,3.465736
86,These HDMI cables are strong and durable. The...,High Quality,5,2,2015-08-27,5.493061
88,It works as intended. Have owned it for over a...,Just buy it. It works and is reliable. 1+ year...,5,1,2015-08-27,3.465736
184,Where should you always buy HDMI cables? Amazo...,Don't go to BestBuy and don't gamble with EBay...,5,1,2015-08-20,3.465736
201,We have always had great success with Amazon B...,We have always had great success with Amazon B...,5,1,2015-08-20,3.465736
...,...,...,...,...,...,...
15328,This is the HDMI cable to get right now: an HD...,A Basic Cable at a Good Price,5,297,2010-11-01,28.485467
15329,I connected my Blu Ray disc player to my HDTV ...,Easy and lightweight HDMI cable does it all.,5,5,2010-11-01,8.958797
15330,"Just unpacked my cable,it is really very high ...",High quality,5,1,2010-11-01,3.465736
15331,I have purchased quite a few AmazonBasics item...,It works.,5,12,2010-11-01,12.824747


In [ ]:
sum(csv_r["star_rating"])/csv_r['star_rating'].count() #Media aritmetica delle star

4.710167612339399

In [ ]:
#media ponderata basata sugli helpful votes
sum(csv_r[csv_r["review_weight"] != 0]["review_weight"])/sum(csv_r[csv_r["review_weight"] != 0].apply(lambda x: math.log(x['helpful_votes']+1), axis=1))

3.9589174045678646

In [ ]:
sum(csv_r[csv_r["review_weight"] != 0]["review_weight"])

2643.4626372768175

In [ ]:
sum(csv_r[csv_r["review_weight"] != 0].apply(lambda x: math.log(x['helpful_votes']+1), axis=1))

667.7236140937789

In [ ]:
#media ponderata basata sugli helpful votes
sum(csv_r['star_rating']*(csv_r['helpful_votes']+0.1))/sum(csv_r['helpful_votes']+0.1) #Adding significance to the ones with 0

4.345601170738329

In [ ]:
#media ponderata basata sugli helpful votes
sum(csv_r['star_rating']*(csv_r['helpful_votes']))/sum(csv_r['helpful_votes']) #Without significance

4.231288343558282

In [ ]:
#media ponderata basata sugli helpful votes scala logaritmica considerando solo quelli con weight != 0
sum(csv_r[csv_r["review_weight"] != 0]["review_weight"])/sum(csv_r[csv_r["review_weight"] != 0].apply(lambda x: math.log(x['helpful_votes']+1), axis=1))

3.9589174045678646

In [ ]:
csv_r

,review_body,review_headline,star_rating,helpful_votes,review_date,review_weight
0,Does the job.,Five Stars,5,0,2015-08-31,0.000000
1,Cool,Five Stars,5,0,2015-08-31,0.000000
2,They work fine.,Five Stars,5,0,2015-08-31,0.000000
3,I love it,Five Stars,5,0,2015-08-31,0.000000
4,"Buenas, excelente cable.",Five Stars,5,0,2015-08-31,0.000000
...,...,...,...,...,...,...
15328,This is the HDMI cable to get right now: an HD...,A Basic Cable at a Good Price,5,297,2010-11-01,28.485467
15329,I connected my Blu Ray disc player to my HDTV ...,Easy and lightweight HDMI cable does it all.,5,5,2010-11-01,8.958797
15330,"Just unpacked my cable,it is really very high ...",High quality,5,1,2010-11-01,3.465736
15331,I have purchased quite a few AmazonBasics item...,It works.,5,12,2010-11-01,12.824747


In [ ]:
#Save the sample into a csv on the shared drive
csv_r.to_csv('/content/drive/Shareddrives/BPM PROJECT/Dataset/sample_weighted.csv', index=False)